In [6]:
import os
import json
import nltk
import math
import numpy as np
from nltk.tokenize import TweetTokenizer

In [7]:
tknzr = TweetTokenizer(reduce_len=True)

videoLength = 28662 # temp const
numberOfMessage = 1276 # temp const

In [8]:
with open("c.txt", "r") as clip_file:
    globalClip = clip_file.read()
    
globalClip = json.loads(globalClip)['data'] 
#videoLength = len(globalClip) # temp const

#print(videoLength) # 28662
#print(globalClip['data'][6296])

In [10]:
#file = open("../TwitchHighlightCrawler/vod/lirik/389178879/Message-" + str(1) + ".json", "r", encoding="utf-8")
#message = file.read()
#message = json.loads(message)['comments'][1]['message']['body']

clipMessage = ''
nonClipMessage = ''

#print(message)
for i in range(1, numberOfMessage + 1):
    with open("../TwitchHighlightCrawler/vod/lirik/389178879/Message-" + str(i) + ".json", "r", encoding="utf-8") as file:
        messages = file.read()
        
    messages = json.loads(messages)['comments']
    for message in messages:
        offset = math.floor( message['content_offset_seconds'] )
        
        if offset >= videoLength:
            break
        
        if (globalClip[offset]): # highlight
            clipMessage += message['message']['body'] + ' '
        else:
            nonClipMessage += message['message']['body'] + ' '
            
''' Tokenization '''
clipTokens = tknzr.tokenize(clipMessage)
nonClipTokens = tknzr.tokenize(nonClipMessage)

''' Convert tokens to NLTK text '''
clipText = nltk.text.Text(clipTokens)
nonClipText = nltk.text.Text(nonClipTokens)

74946
185181


In [8]:
def lexical_diversity(text): # base diversity measure
    return len(set(text)) / len(text)

In [4]:
def shannon_entropy(text): # entropy diversity measure
    entropy = 0
    
    vocabulary = set(text)
    textLength = len(text)
    for word in vocabulary:
        p = text.count(word) / textLength
        
        entropy -= p * math.log2(p)
        
    return entropy

In [9]:
def normalized_shannon_entropy(text): # entropy diversity measure (normalized)
    entropy = 0
    
    textLength = len(text)
    if textLength <= 1:
        return entropy # 0
    else:
        vocabulary = set(text)
        for word in vocabulary:
            p = text.count(word) / textLength

            entropy -= p * math.log2(p)

        return entropy / math.log2(textLength)

In [10]:
#messageEachSecond = np.full((videoLength), '')
messageEachSecond = [''] * videoLength

for i in range(1, numberOfMessage + 1):
    with open("../TwitchHighlightCrawler/vod/lirik/389178879/Message-" + str(i) + ".json", "r", encoding="utf-8") as file:
        messages = file.read()
        
    messages = json.loads(messages)['comments']
    for message in messages:
        offset = math.floor( message['content_offset_seconds'] )
        
        if offset >= videoLength:
            break
        
        if messageEachSecond[offset]:
            messageEachSecond[offset] += ' ' + message['message']['body']
        else:
            messageEachSecond[offset] = message['message']['body']

# Clip Local

In [11]:
clipList = os.listdir('../TwitchHighlightCrawler/vod/lirik/389178879/clip')
diversity = 0
diversities = list()

for filename in clipList:
    with open('../TwitchHighlightCrawler/vod/lirik/389178879/clip/' + filename, "r", encoding="utf-8") as file:
        data = file.read()
        
    data = json.loads(data)
    offset = data['vod']['offset']
    duration = math.ceil(data['duration'])
    
    clipMessage = ''
    for i in range(offset, offset+duration):
        if not messageEachSecond[i]: # no message in this second
            continue
            
        clipMessage += messageEachSecond[i] + ' '
        
    clipTokens = tknzr.tokenize(clipMessage) # tokenization
    clipText = nltk.text.Text(clipTokens) # convert tokens to NLTK text
    
    #score = shannon_entropy(clipText)
    score = normalized_shannon_entropy(clipText)
    diversities.append(score)
    diversity += score

In [12]:
#print(diversity / len(clipList)) # 6.358026626174814 / 0.7793819629954689
#print(diversity)

0.7793819629954689


In [14]:
'''max_score = 0
for score in diversities:
    if score > max_score:
        max_score = score
        
print(max_score) # 7.785621252073685


clip_distribution = np.zeros(math.floor(7.785621252073685 * 10) + 1, np.int64)
for score in diversities:
    clip_distribution[ math.floor(score * 10) ] += 1

print(clip_distribution)'''

clip_distribution = np.zeros(101, np.int64)
for score in diversities:
    clip_distribution[ math.floor(score * 100) ] += 1

print(clip_distribution)

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  1  0  2  1  1  4
  2  0  1  0  2  1  1  0  1  0  5  4  1  3 10  2  1  3  1  3  8 14 16 16
 27 20 21 34 38 33 83 61 34 24 26 22 17 27 18 11 16 11  7 16  0  6  3  1
  2  1  1  0  1]


# General Distribution (45 min / 1s)

In [20]:
diversities = list()

for i in range(0, videoLength-4):
    localMessage = ''
    
    for j in range(i, i+5):
        if not messageEachSecond[j]: # no message in this second
            continue
            
        localMessage += messageEachSecond[j] + ' '
        
    localTokens = tknzr.tokenize(localMessage) # tokenization
    localText = nltk.text.Text(localTokens) # convert tokens to NLTK text
    
    score = shannon_entropy(localText)
    diversities.append(score)
    
    for j in range(i+5, i+60):
        if j >= videoLength: # over video length
            break
        
        if not messageEachSecond[j]: # no message in this second
            diversities.append(diversities[-1]) # score is same as last second
            continue
        else:
            localMessage += messageEachSecond[j] + ' '
            
            localTokens = tknzr.tokenize(localMessage) # tokenization
            localText = nltk.text.Text(localTokens) # convert tokens to NLTK text
            
            score = shannon_entropy(localText)
            diversities.append(score)

In [22]:
'''max_score = 0
for score in diversities:
    if score > max_score:
        max_score = score
        
print(len(diversities)) # 1603308
print(max_score) # 8.315601513754167'''

clip_distribution = np.zeros(math.floor(8.315601513754167 * 10) + 1, np.int64)
for score in diversities:
    clip_distribution[ math.floor(score * 10) ] += 1

print(clip_distribution)

[  420     0     0     1     0     0     3     8    33    62   254     5
    47    79    18   397    29    91    77   198   399   214   268   529
   358   889   606   743  1434  1389  1953  2653  2607  3417  4297  4621
  4621  6003  6105  6583  7721  8215  9675 10669  9806 10765 11892 13023
 13918 15486 17625 19757 21619 23435 25580 28359 30892 33831 37988 40975
 45117 49696 55406 60082 66442 71395 75747 80064 82658 84053 83190 79484
 76320 65760 54675 43655 30974 24035 16339  9374  4487  1423   276    14]


In [15]:
diversities1 = list() # nonoverlap
diversities2 = list() # full overlap (multiple situations)
diversities3 = list() # partial overlap

i = 0
while i < videoLength:
    localMessage = ''
    
    nonOverlapFlag = True
    fullOverlapFlag = True
    for j in range(i, i+30):
        if j >= videoLength: # over video length
            break
        
        if not messageEachSecond[j]: # no message in this second
            continue
        
        if globalClip[j]: # overlap
            nonOverlapFlag = False
        else:
            fullOverlapFlag = False
        localMessage += messageEachSecond[j] + ' '
        
    localTokens = tknzr.tokenize(localMessage) # tokenization
    localText = nltk.text.Text(localTokens) # convert tokens to NLTK text
    
    #score = shannon_entropy(localText)
    score = normalized_shannon_entropy(localText)
    if nonOverlapFlag:
        diversities1.append(score)
    elif fullOverlapFlag:
        diversities2.append(score)
    else:
        diversities3.append(score)
    
    i += 30

In [16]:
'''max_score = 0
for score in diversities3:
    if score > max_score:
        max_score = score
        
print(len(diversities3)) # diversities1: 583 / diversities2: 89 / diversities3: 284
print(max_score) # diversities1: 7.756178018870385 / diversities2: 7.642048407314907 / diversities3: 7.672029025582002

clip_distribution1 = np.zeros(math.floor(7.756178018870385 * 10) + 1, np.int64)
for score in diversities1:
    clip_distribution1[ math.floor(score * 10) ] += 1

print(clip_distribution1)

clip_distribution2 = np.zeros(math.floor(7.642048407314907 * 10) + 1, np.int64)
for score in diversities2:
    clip_distribution2[ math.floor(score * 10) ] += 1

print(clip_distribution2)

clip_distribution3 = np.zeros(math.floor(7.6720290255820027 * 10) + 1, np.int64)
for score in diversities3:
    clip_distribution3[ math.floor(score * 10) ] += 1

print(clip_distribution3)'''

clip_distribution1 = np.zeros(101, np.int64)
for score in diversities1:
    clip_distribution1[ math.floor(score * 100) ] += 1

print(clip_distribution1)

clip_distribution2 = np.zeros(101, np.int64)
for score in diversities2:
    clip_distribution2[ math.floor(score * 100) ] += 1

print(clip_distribution2)

clip_distribution3 = np.zeros(101, np.int64)
for score in diversities3:
    clip_distribution3[ math.floor(score * 100) ] += 1

print(clip_distribution3)

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  1  0  0  2  0  2  1  0  1  0  1  1  1  0  1
  0  2  0  2  1  1  2  2  0  0  1  1  1  0  1  0  0  0  1  0  5  3  4  8
  1  4  4 10  8 12 10 14 14 15 25 33 22 35 36 43 46 48 47 45 29 11 15  5
  3  1  1  0  0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 2 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 5 1 3
 0 4 4 2 5 6 5 4 1 3 7 4 4 5 5 5 3 3 1 0 1 1 0 0 0 0 0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  1  0  1  1  1  0  0  0  0  0  0  0  1  1  0  0  2  0  1  0
  1  0  0  3  1  0  1  1  0  0  0  1  0  0  0  2  1  0  3  1  5  1  6  2
  2  4  4  3  6 11  9  6  9  9 19 13 14 14 23 20 17 12 10 14 14  7  0  4
  1  1  0  0  0]


In [26]:
t = 'aaa bbb cccc dddd eeee ffff gggg'
tt = tknzr.tokenize(t)
ttt = nltk.text.Text(tt)

2.807354922057604
2.807354922057604


In [29]:
#print(lexical_diversity(clipTokens))
#print(lexical_diversity(nonClipTokens))
#print(len(set(clipTokens)))
#print(len(clipTokens))
#print(len(set(nonClipTokens)))
#print(len(nonClipTokens))

In [30]:
#print(clipTokens)

In [32]:
print(shannon_entropy(clipText))
print(shannon_entropy(nonClipText))

9.900873478937825
10.338312052933961
